In [16]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

from collections import Counter
from sklearn.metrics import classification_report


In [17]:
df = pd.read_csv('/Users/awesome/Downloads/предсказание_спроса/train.csv', sep=';')
df.head()

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
0,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00101,0.00103,...,0.13027,0.0,0.00000,NaN,NaN,NaN,NaN,NaN,0.01737,0.0
1,1,2020-05-01,Город,Москва,Москва,8611aa01fffffff,55.975851,37.237085,0.00000,0.00027,...,0.08756,0.0,0.00000,NaN,NaN,NaN,NaN,NaN,0.01152,0.0
2,1,2020-05-01,Город,Москва,Москва,861181b6fffffff,55.622721,37.695121,0.00339,0.00313,...,0.09243,0.0,0.00000,0.11053,0.57895,0.00526,0.02105,0.00000,0.01540,0.0
3,1,2020-05-01,Город,Москва,Москва,8611aa017ffffff,55.941586,37.157487,0.00048,0.00054,...,0.10192,0.0,0.00049,NaN,NaN,NaN,NaN,NaN,0.01495,0.0
4,1,2020-05-01,Город,Москва,Москва,8611aa637ffffff,55.797494,37.676200,0.00164,0.00179,...,0.09620,0.0,0.00000,0.14444,0.64444,0.01111,0.04444,0.01111,0.01266,0.0


In [18]:
df.subject_name.value_counts()

Москва             22606
Санкт-Петербург    15876
Краснодарский      11140
Московская         10994
Нижегородская      10371
                   ...  
Алтай                208
Тыва                 168
Ингушетия            145
Ненецкий             101
Чукотский              2
Name: subject_name, Length: 82, dtype: int64

In [19]:
df.city_name.value_counts()

Москва             22444
Санкт-Петербург    13805
Новосибирск         5018
Нижний Новгород     4785
Тольятти            3616
                   ...  
Сретенское             1
Усолье                 1
Карандашово            1
Трубный                1
Сюля                   1
Name: city_name, Length: 4633, dtype: int64

In [20]:
df.label.value_counts()

0    287152
1      7106
Name: label, dtype: int64

In [21]:
df.period = pd.to_datetime(df.period)

df['year'] = df.period.dt.year
df['month'] = df.period.dt.month
df['quarter_of_year'] = df.period.dt.quarter

df = df.drop(columns=['period'])

In [22]:
# df = df[['label', 'f12', 'f10', 'f3', 'f28', 'f11', 'subject_name',
#          'f15', 'f1', 'f20', 'f2', 'hex_lat', 'f21',
#          'f17', 'month', 'hex_lon', 'f6', 'f9', 'f24',
#          'f16', 'subject_type', 'quarter_of_year']]

In [23]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

df = df.join(pd.DataFrame(lb.fit_transform(df['subject_type']),
                                  columns=lb.classes_, 
                                  index=df.index))

df = df.join(pd.DataFrame(lb.fit_transform(df['subject_name']),
                                  columns=lb.classes_, 
                                  index=df.index))

df = df.drop(columns=['subject_name', 'subject_type', 'city_name', 'hex'])

In [26]:
# df = df.fillna(0)

In [27]:
X = df.drop("label", axis=1)
y = df["label"]

In [28]:
from numpy import hstack
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
import lightgbm as lgb

def get_models():
    models = []
    models.append(('lr', CatBoostClassifier(n_estimators=350)))
    models.append(('gbr', XGBClassifier(n_estimators=350)))
    models.append(('cbr', lgb.LGBMClassifier(n_estimators=350)))

    return models

# обучает ансамблевую модель с использованием обучающего 
# и независимого набора данных
def fit_ensemble(models, X_train, X_val, y_train, y_val):
    meta_X = list()
    for name, model in models:
        # обучение
        model.fit(X_train, y_train) 
        # предсказания для каждой модели
        reg = model.predict(X_val)
        # в матрицу с 1 колонкой
        reg = reg.reshape(len(reg), 1)
        # прогнозные значения храним в списке (потом сделаем массив)
        meta_X.append(reg)

    # создаем 2д массив
    meta_X = hstack(meta_X)
    # определяем нашу смешенную модель
    blender_model = lgb.LGBMClassifier(n_estimators=300)
    # обучение метамодели
    blender_model.fit(meta_X, y_val)

    return blender_model

# собрать прогнозы каждой базовой модели в обучающий набор данных,
# сложить прогнозы вместе и вызвать predict() в ансамблевой модели 
def predict_ensemble(models, blender_model, X_test):
    # предсказания с базовыми моделями
    meta_X = list()
    for name, model in models:
        # предсказание
        reg = model.predict(X_test)
        # в матрицу с 1 колонкой
        reg = reg.reshape(len(reg), 1)
        meta_X.append(reg)
    # создаем 2д массив из предсказаний
    meta_X = hstack(meta_X)
    # predict
    return blender_model.predict(meta_X)

In [29]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, 
                                                              test_size=0.15, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full,
                                                  test_size=0.15, random_state=1) 

# базовая модель
models = get_models()
# обучение
blender_model = fit_ensemble(models, X_train, X_val, y_train, y_val)
# тестирование
prediction = predict_ensemble(models, blender_model, X_test)
# метрики

Learning rate set to 0.266008
0:	learn: 0.3360742	total: 75.5ms	remaining: 26.4s
1:	learn: 0.1954634	total: 89.1ms	remaining: 15.5s
2:	learn: 0.1391451	total: 103ms	remaining: 11.9s
3:	learn: 0.1182406	total: 115ms	remaining: 9.96s
4:	learn: 0.1074505	total: 130ms	remaining: 8.99s
5:	learn: 0.1024538	total: 143ms	remaining: 8.21s
6:	learn: 0.0998201	total: 156ms	remaining: 7.66s
7:	learn: 0.0978217	total: 170ms	remaining: 7.26s
8:	learn: 0.0967349	total: 182ms	remaining: 6.91s
9:	learn: 0.0954369	total: 196ms	remaining: 6.65s
10:	learn: 0.0947610	total: 214ms	remaining: 6.58s
11:	learn: 0.0945001	total: 229ms	remaining: 6.44s
12:	learn: 0.0941279	total: 243ms	remaining: 6.3s
13:	learn: 0.0937801	total: 256ms	remaining: 6.14s
14:	learn: 0.0934691	total: 269ms	remaining: 6s
15:	learn: 0.0932856	total: 281ms	remaining: 5.88s
16:	learn: 0.0930851	total: 294ms	remaining: 5.76s
17:	learn: 0.0928733	total: 306ms	remaining: 5.65s
18:	learn: 0.0927504	total: 318ms	remaining: 5.54s
19:	learn: 0.

In [30]:
print(classification_report(y_test, prediction))

print(Counter(prediction).values())
print(Counter(y_test).values())

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     43091
           1       0.40      0.03      0.06      1048

    accuracy                           0.98     44139
   macro avg       0.69      0.52      0.52     44139
weighted avg       0.96      0.98      0.97     44139

dict_values([44055, 84])
dict_values([43091, 1048])


In [31]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, prediction)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[43041    50]
 [ 1014    34]]

True Positives(TP) =  43041

True Negatives(TN) =  34

False Positives(FP) =  50

False Negatives(FN) =  1014
